In [1]:
import os
pwd_list = os.getcwd().split('\\')
if pwd_list[-1] != 'RiskAI' and 'RiskAI' in pwd_list:
    os.chdir('/'.join(pwd_list[:pwd_list.index('RiskAI')+1]))

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from preprocess import get_datas

In [73]:
def mapa_calor(df):
    df_corr = df.corr()[['GSRAII Index', 'GSRAII_diff', exog]]
    df_corr = df_corr.loc[[idx for idx in df_corr.index if 
                          idx not in ['GSRAII Index', 'GSRAII_diff', exog]]]
    fig = px.imshow(df_corr.T, width=50*len(df_corr), height=600, color_continuous_scale='RdBu', zmin=-1, zmax=1)
    
    for i, col in enumerate(df_corr.columns):
        best_corr = df_corr.iloc[np.argmax(df_corr[col].abs())]
        print(f'MAIOR CORREL: {col} X {best_corr.name}: {best_corr[col]:.4f} ')

    return fig

# Carregamento dos dados
- Dados foram colocados na mesma escala com MinMaxScaler
- Variavel exogena foram avaliado os valores com lag, pois é a informação disponivel para a variavel target 

In [52]:
scaler = MinMaxScaler()
df = get_datas(return_df=True, scaler=scaler)

In [53]:
exog = 'acwi'
df = df[['GSRAII Index', 'GSRAII_diff', 'acwi_log_diff']]
df.rename(columns={'acwi_log_diff':exog}, inplace=True)

# Analise lag

In [68]:
df_lag = df.copy()

for i in range(1, 11):
    df_lag[f'{exog}_lag_{i}'] = df_lag[exog].shift(i).values

In [74]:
mapa_calor(df_lag)

MAIOR CORREL: GSRAII Index X acwi_lag_2: 0.2389 
MAIOR CORREL: GSRAII_diff X acwi_lag_1: -0.1063 
MAIOR CORREL: acwi X acwi_lag_6: 0.0752 


# Analise Médias Móveis

In [56]:
'lag' in 'lag_ioi'

True

In [65]:
df_medmov = df_lag.copy()
cols_lag = [col for col in df_medmov.columns if 'lag' in col]

for lag in cols_lag:
    for i in range(2, 30):
        df_medmov[f'{lag}_medmov_{i}'] = df_medmov[f'{lag}'].rolling(i).mean()

In [75]:
mapa_calor(df_medmov.drop(columns=cols_lag))

MAIOR CORREL: GSRAII Index X acwi_lag_1_medmov_20: 0.7010 
MAIOR CORREL: GSRAII_diff X acwi_lag_1_medmov_19: -0.1158 
MAIOR CORREL: acwi X acwi_lag_10_medmov_13: 0.0508 


In [48]:
df_medmov.columns

Index(['GSRAII Index', 'GSRAII_diff', 'acwi', 'acwi_lag_1', 'acwi_lag_2',
       'acwi_lag_3', 'acwi_lag_4', 'acwi_lag_5', 'acwi_lag_6', 'acwi_lag_7',
       ...
       'acwi_lag_10_medmov_20', 'acwi_lag_10_medmov_21',
       'acwi_lag_10_medmov_22', 'acwi_lag_10_medmov_23',
       'acwi_lag_10_medmov_24', 'acwi_lag_10_medmov_25',
       'acwi_lag_10_medmov_26', 'acwi_lag_10_medmov_27',
       'acwi_lag_10_medmov_28', 'acwi_lag_10_medmov_29'],
      dtype='object', length=293)

In [13]:
# std, diferenca de media, IFR, MACD